In [1]:
from pj import *

In [2]:
class AN(enum.Enum):
    PLUS, MINUS, PUTA, PODIJELJENO, NA, OTVORENA, ZATVORENA = '+-*/^()'
    VECE, MANJE, JEDNAKO = '><='
    VECEJEDNAKO, MANJEJEDNAKO, RAZLICITO = '>=', '<=', '!='
    class BROJ(Token):
        def vrijednost(self): return int(self.sadržaj)
        def prevedi(self): yield 'PUSH', self.vrijednost()

In [3]:
def an_lex(izraz):
    lex = Tokenizer(izraz)
    for znak in iter(lex.čitaj, ''):
        if znak.isdigit():
            if znak != '0': lex.zvijezda(str.isdigit)
            yield lex.token(AN.BROJ)
        else: yield lex.literal(AN)

In [4]:
### Beskontekstna gramatika: (desno asocirani operatori)
# izraz -> član (PLUS | MINUS | VECE | MANJE | JEDNAKO) izraz | član
# član -> faktor PUTA član | faktor PODIJELJENO član | faktor
# faktor -> baza NA faktor | baza
# baza -> BROJ | OTVORENA izraz ZATVORENA

In [5]:
### Apstraktna sintaksna stabla
# Zbroj: pribrojnici
# Razlika: umanjenici
# Umnožak: faktori
# Količnik: djeljenici
# Potencija: baza eksponent

In [6]:
class ANParser(Parser):
    def izraz(self):
        prvi = self.član()
        if self >> AN.PLUS:
            drugi = self.izraz()
            return Zbroj([prvi, drugi])
        elif self >> AN.MINUS:
            drugi = self.izraz()
            return Razlika([prvi, drugi])
        elif self >> AN.VECE:
            drugi = self.izraz()
            return UsporedbaVece([prvi, drugi])
        elif self >> AN.MANJE:
            drugi = self.izraz()
            return UsporedbaManje([prvi, drugi])
        elif self >> AN.JEDNAKO:
            drugi = self.izraz()
            return UsporedbaJednako([prvi, drugi])
        else:
            return prvi

    def član(self):
        faktor = self.faktor()
        if self >> AN.PUTA: return Umnožak([faktor, self.član()])
        elif self >> AN.PODIJELJENO: return Količnik([faktor, self.član()])
        else: return faktor

    def faktor(self):
        baza = self.baza()
        if self >> AN.NA: return Potencija(baza, self.faktor())
        else: return baza

    def baza(self):
        if self >> AN.BROJ: return self.zadnji
        elif self >> AN.OTVORENA:
            u_zagradi = self.izraz()
            self.pročitaj(AN.ZATVORENA)
            return u_zagradi
        else: raise self.greška()

    start = izraz

In [7]:
nula = Token(AN.BROJ, '0')
jedan = Token(AN.BROJ, '1')

In [8]:
class Zbroj(AST('pribrojnici')):
    def vrijednost(izraz):
        a, b = izraz.pribrojnici
        return a.vrijednost() + b.vrijednost()

    def prevedi(izraz):
        a, b = izraz.pribrojnici
        yield from a.prevedi()
        yield from b.prevedi()
        yield 'ADD',

In [9]:
class Razlika(AST('umanjenici')):
    def vrijednost(izraz):
        a, b = izraz.umanjenici
        return a.vrijednost() - b.vrijednost()

    def prevedi(izraz):
        a, b = izraz.umanjenici
        yield from a.prevedi()
        yield from b.prevedi()
        yield 'SUB',

In [10]:
class UsporedbaVece(AST('sadrzaj')):
    def vrijednost(izraz):
        a, b = izraz.sadrzaj
        if (int(a.vrijednost())>int(b.vrijednost())): return a.vrijednost()
        else: raise SemantičkaGreška(int(a.vrijednost()), 'nije veći od', int(b.vrijednost()))

    def prevedi(izraz):
        a, b = izraz.sadrzaj
        yield from a.prevedi()
        yield from b.prevedi()
        yield 'U>',

In [11]:
class UsporedbaManje(AST('sadrzaj')):
    def vrijednost(izraz):
        a, b = izraz.sadrzaj
        if (int(a.vrijednost())<int(b.vrijednost())): return a.vrijednost()
        else: raise SemantičkaGreška(int(a.vrijednost()), 'nije manji od', int(b.vrijednost()))

    def prevedi(izraz):
        a, b = izraz.sadrzaj
        yield from a.prevedi()
        yield from b.prevedi()
        yield 'U<',

In [12]:
class UsporedbaJednako(AST('sadrzaj')):
    def vrijednost(izraz):
        a, b = izraz.sadrzaj
        if (int(a.vrijednost())==int(b.vrijednost())): return a.vrijednost()
        else: raise SemantičkaGreška('brojevi nisu isti')

    def prevedi(izraz):
        a, b = izraz.sadrzaj
        yield from a.prevedi()
        yield from b.prevedi()
        yield 'U=',

In [13]:
class Umnožak(AST('faktori')):
    def vrijednost(izraz):
        a, b = izraz.faktori
        return a.vrijednost() * b.vrijednost()

    def prevedi(izraz):
        a, b = izraz.faktori
        yield from a.prevedi()
        yield from b.prevedi()
        yield 'MUL',

In [14]:
class Količnik(AST('djeljenici')):
    def vrijednost(izraz):
        a, b = izraz.djeljenici
        if (int(b.vrijednost())!=0): return a.vrijednost() / b.vrijednost()
        else: raise SemantičkaGreška('nema dijeljenja s nulom')

    def prevedi(izraz):
        a, b = izraz.djeljenici
        yield from a.prevedi()
        yield from b.prevedi()
        yield 'DIV',

In [15]:
class Potencija(AST('baza eksponent')):
    def vrijednost(izraz):
        return izraz.baza.vrijednost() ** izraz.eksponent.vrijednost()

    def prevedi(izraz):
        yield from izraz.baza.prevedi()
        yield from izraz.eksponent.prevedi()
        yield 'POW',


In [16]:
def testiraj(izraz):
    stablo = ANParser.parsiraj(an_lex(izraz))
    print(stablo)
    mi = stablo.vrijednost()
    print(izraz, 'mi:', mi)
    for instrukcija in stablo.prevedi(): print('\t', *instrukcija)
    print("-----------------------------------------------------")

In [17]:
    testiraj('(3-3)/3^1')
    testiraj('2^0^0^0^0')
    testiraj('2+(0+1*1*2)')
    with očekivano(LeksičkaGreška): testiraj('2 3')
    with očekivano(SintaksnaGreška): testiraj('2+')
    with očekivano(SemantičkaGreška): testiraj('2/0')

Količnik(djeljenici=[Razlika(umanjenici=[BROJ'3', BROJ'3']), Potencija(baza=BROJ'3', eksponent=BROJ'1')])
(3-3)/3^1 mi: 0.0
	 PUSH 3
	 PUSH 3
	 SUB
	 PUSH 3
	 PUSH 1
	 POW
	 DIV
-----------------------------------------------------
Potencija(baza=BROJ'2', eksponent=Potencija(baza=BROJ'0', eksponent=Potencija(baza=BROJ'0', eksponent=Potencija(baza=BROJ'0', eksponent=BROJ'0'))))
2^0^0^0^0 mi: 2
	 PUSH 2
	 PUSH 0
	 PUSH 0
	 PUSH 0
	 PUSH 0
	 POW
	 POW
	 POW
	 POW
-----------------------------------------------------
Zbroj(pribrojnici=[BROJ'2', Zbroj(pribrojnici=[BROJ'0', Umnožak(faktori=[BROJ'1', Umnožak(faktori=[BROJ'1', BROJ'2'])])])])
2+(0+1*1*2) mi: 4
	 PUSH 2
	 PUSH 0
	 PUSH 1
	 PUSH 1
	 PUSH 2
	 MUL
	 MUL
	 ADD
	 ADD
-----------------------------------------------------
LeksičkaGreška: Redak 1, stupac 2: neočekivan znak ' '
SintaksnaGreška: Redak zadnji, stupac 0: neočekivan token KRAJ
Očekivano: BROJ ili OTVORENA
Količnik(djeljenici=[BROJ'2', BROJ'0'])
SemantičkaGreška: nema dijelj

In [22]:
testiraj('(2+1)>3')

UsporedbaVece(sadrzaj=[Zbroj(pribrojnici=[BROJ'2', BROJ'1']), BROJ'3'])


SemantičkaGreška: (3, 'nije veći od ', 3)